<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9C%A0%EC%98%88%EC%8A%AC_yogiyo%EB%A9%80%ED%8B%B0%EB%9D%BC%EB%B2%A8%2B_%EA%B8%8D%EB%B6%80%EC%A0%95_%EB%B6%84%EC%84%9D%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. 멀티라벨 모델 (review)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [ ]:
# yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv") #18686개

# yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean.csv") #24998개

yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908.csv") #24998개


In [ ]:
yogiyo

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,[직화]삼겹옛날도시락
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요:),5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치"
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥"
3,1988응답하라추억의옛날도시락-방이점,"..,........",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,삼겹정찬도시락
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러..."
...,...,...,...,...,...,...,...,...
37135,짬뽕관-전주금암점,맛있어요 ㅎ저번에주문하고 맛있어서또주문핰건데\n늦은시간에시켜서그렄지 국물이 엄청 걸...,5,5.0,5.0,5.0,고객님 안녕하세요 짬뽕관입니다~~^-^\n소중한 리뷰 너무 감사합니다!!🙂\n재주문...,짬뽕 ＋ 탕수육 set
37136,짬뽕관-전주금암점,맛있어요 또시킬게요,5,5.0,5.0,5.0,고객님 안녕하요 짬뽕관입니다!!^_^\n소중한 리뷰 남겨주셔 서 정말 달감사합니다!...,순두부짬뽕
37137,짬뽕관-전주금암점,정말 맛있게 잘먹었어요!!자주 시켜 먹을께요!,5,5.0,5.0,5.0,고객님 안녕하세요 짬뽕관입니다~~^_^\n맛있게 드셨다는 말씀에 힘이 나는거 같습니...,콩국수
37138,짬뽕관-전주금암점,와우.. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 ...,5,5.0,5.0,5.0,고객님 안녕하세요 짬뽕관입니다!!^_^\n맛있게 드셔주시고 정성스러운 사진 리뷰까지...,"차슈짬뽕,마파두부밥"


In [ ]:
# google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

In [ ]:
# yogiyo_added =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905.csv")


In [ ]:
train= yogiyo[['고객리뷰','사장댓글','별점']]

# train = yogiyo[['review', 'reply','rating']]

rename_cols = ['review', 'reply','rating']
train.columns = rename_cols
train.head()

,review,reply,rating
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5
1,기름 있는 부분이 생각보다 많아요! 참고하세요:),소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5
3,"..,........",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5


In [ ]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

<ipython-input-8-6374a3c305ac>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = review_list


,review,reply,rating
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5


In [ ]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

<ipython-input-9-2764c488e75c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = reply_list


,review,reply,rating
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5


## 키워드 추출 (Keybert + Kiwi)
- 리뷰별로 kiwi로 추출한 형태소 (명사류, 형용사, 어근)에서 keybert 키워드 추출
- keybert 키워드 pool 중에서 top 100 키워드 선정, 직접 5개 카테고리('맛, 양, 가격, 서비스, 배달')에 관한 키워드를 추출함
- 각 리뷰별로 kiwi 분석 결과에 키워드가 있는지에 따라 5개 카테고리 멀티 분류

In [ ]:
!pip install kiwipiepy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 32.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=5fbe981104faefe24bdb91f4e091cb42330dae5349af94896d553878934f059a
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [ ]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=d9bc503f4d21ba06a91df0e0c62ed39b4fba3ee2fbe81974e229cc8b07b0248b
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=943fd2237ad6f8a6543618984ed8a5f061e538dd97414f4d00632355cb30873c
 

In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()


- 키워드 품사 확인

In [ ]:
kiwi.analyze('고기 양도 많고 구성이 알차요')  # 양 == NNG / 알차 == VA

[([Token(form='고기', tag='NNG', start=0, len=2),
   Token(form='양', tag='NNG', start=3, len=1),
   Token(form='도', tag='JX', start=4, len=1),
   Token(form='많', tag='VA', start=6, len=1),
   Token(form='고', tag='EC', start=7, len=1),
   Token(form='구성', tag='NNG', start=9, len=2),
   Token(form='이', tag='JKS', start=11, len=1),
   Token(form='알차', tag='VA', start=13, len=2),
   Token(form='어요', tag='EF', start=14, len=2)],
  -62.784297943115234)]

In [ ]:
kiwi.analyze('정말 맛있어요!')  # 맛 == VA (맛있)

[([Token(form='정말', tag='MAG', start=0, len=2),
   Token(form='맛있', tag='VA', start=3, len=2),
   Token(form='어요', tag='EF', start=5, len=2),
   Token(form='!', tag='SF', start=7, len=1)],
  -14.709687232971191)]

In [ ]:
kiwi.analyze('가격이 너무 비싸요ㅠㅠ 더 싸고 가성비 좋으면 많이 시켜먹을 듯') # 가격, 가성비 == NNG / 비싸 == VA / 싸 == VV

[([Token(form='가격', tag='NNG', start=0, len=2),
   Token(form='이', tag='JKS', start=2, len=1),
   Token(form='너무', tag='MAG', start=4, len=2),
   Token(form='비싸', tag='VA', start=7, len=2),
   Token(form='어요', tag='EF', start=8, len=2),
   Token(form='ㅠㅠ', tag='SW', start=10, len=2),
   Token(form='더', tag='MAG', start=13, len=1),
   Token(form='싸', tag='VA', start=15, len=1),
   Token(form='고', tag='EC', start=16, len=1),
   Token(form='가성비', tag='NNG', start=18, len=3),
   Token(form='좋', tag='VA', start=22, len=1),
   Token(form='으면', tag='EC', start=23, len=2),
   Token(form='많이', tag='MAG', start=26, len=2),
   Token(form='시키', tag='VV', start=29, len=2),
   Token(form='어', tag='EC', start=30, len=1),
   Token(form='먹', tag='VV', start=31, len=1),
   Token(form='을', tag='ETM', start=32, len=1),
   Token(form='듯', tag='NNB', start=34, len=1)],
  -116.14079284667969)]

In [ ]:
train['review']

0                                맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요
1                               기름 있는 부분이 생각보다 많아요! 참고하세요 
2          양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!
3                                                      .,.
4        냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...
                               ...                        
37135    맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...
37136                                           맛있어요 또시킬게요
37137                             정말 맛있게 잘먹었어요!자주 시켜 먹을께요!
37138    와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...
37139    제발 양파추가 선택란 하나 만들어주세요. 단무지대신 양파달라고 요청써놨지만 바쁠때 ...
Name: review, Length: 37140, dtype: object

In [ ]:
kiwi_reviews = []

for review in tqdm(train['review']):
    results = []  # 결과 리스트를 각 리뷰마다 초기화
    result = kiwi.analyze(review)
    for token, pos, _, _ in result[0][0]:
        if pos in ['NNG', 'VA', 'VV', 'XR']:  # 일반명사, 형용사, 동사, 어근
            results.append(token)
    kiwi_reviews.append(results)


  0%|          | 0/37140 [00:00<?, ?it/s]

In [ ]:
train['kiwi_reviews'] = kiwi_reviews

<ipython-input-18-27476ffd4693>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['kiwi_reviews'] = kiwi_reviews


In [ ]:
# train['kiwi_reviews'].to_csv(f'{DATA_PATH}kiwi_reviews.csv', index=False)

In [ ]:
# kiwi_reviews = pd.read_csv(f"{DATA_PATH}kiwi_reviews.csv")

In [ ]:
# train = pd.concat([train,kiwi_reviews],axis=1)

In [ ]:
train.head(10)

,review,reply,rating,kiwi_reviews
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,"[맛있, 먹, 묵사발, 시원, 맛있]"
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,"[기름, 있, 부분, 생각, 많, 참, 하]"
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,"[양, 푸짐, 맛있, 청국장, 냄새, 나, 콩, 들, 좋]"
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5,[]
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5,"[냉면, 시키, 미니, 냉면, 합치, 양, 적, 보이, 김치, 정식, 양, 적, 보..."
5,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께 해주셔서 영광입니다. 다음에도 고객님의 즐거운 순...,5,"[맛나, 묵]"
6,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,"[맛, 양, 만족, 배달, 보내, 만족]"
7,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,"저희 메뉴 맛있죠? 이렇게 빠져 드시는 거에요 . 그냥 일로서 만드는 게 아니라, ...",5,"[모르, 은근, 시키, 먹, 먹]"
8,김치찌개 된장찌개 잡채 다 잘먹었습니다,전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...,5,"[김치찌개, 된장찌개, 잡채, 먹]"
9,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요. 다음에도 또시킬듯요,안녕하세요~! D 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주...,5,"[처음, 맛있, 리뷰, 서비스, 괜찮, 다음, 시키]"


In [ ]:
# from keybert import KeyBERT

# keybert_model = KeyBERT()

In [ ]:

# keybert_reviews = []
# for keyword in tqdm(train['kiwi_reviews']):
#     tmp = []
#     keyword = ' '.join(keyword)
#     result = keybert_model.extract_keywords(keyword,
#                               keyphrase_ngram_range=(1, 1),
#                               use_mmr=True,
#                               diversity=0.7,
#                               stop_words=None,
#                               top_n=5)
#     tmp.append(result)
#     keybert_reviews.extend(tmp)

In [ ]:
# train['keybert_reviews'] = keybert_reviews

In [ ]:
# train['keybert_reviews'].to_csv(f"{DATA_PATH}keybert_reviews.csv", index=False)
# train['keybert_reviews'].to_csv(f"{DATA_PATH}keybert_reviews_24000.csv", index=False)
# train['keybert_reviews'].to_csv(f"{DATA_PATH}keybert_reviews_37000.csv", index=False)

In [ ]:
# keybert_reviews = pd.read_csv(f"{DATA_PATH}keybert_reviews.csv")
# keybert_reviews = pd.read_csv(f"{DATA_PATH}keybert_reviews_24000.csv")
keybert_reviews = pd.read_csv(f"{DATA_PATH}keybert_reviews_37000.csv")

In [ ]:
train = pd.concat([train,keybert_reviews],axis=1)

In [ ]:
train

,review,reply,rating,kiwi_reviews,keybert_reviews
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,"[맛있, 먹, 묵사발, 시원, 맛있]","[('묵사발', 0.8007), ('시원', 0.6677), ('맛있', 0.4334)]"
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,"[기름, 있, 부분, 생각, 많, 참, 하]","[('생각', 0.7255), ('기름', 0.5717), ('부분', 0.5354)]"
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,"[양, 푸짐, 맛있, 청국장, 냄새, 나, 콩, 들, 좋]","[('청국장', 0.6928), ('푸짐', 0.3954), ('냄새', 0.315..."
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5,[],[]
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5,"[냉면, 시키, 미니, 냉면, 합치, 양, 적, 보이, 김치, 정식, 양, 적, 보...","[('정식', 0.4585), ('시키', 0.3429), ('합치', 0.323)..."
...,...,...,...,...,...
37135,맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...,고객님 안녕하세요 짬뽕관입니다~ 소중한 리뷰 너무 감사합니다! 재주문해 주셨는데 국...,5,"[맛있, 저번, 주문, 맛있, 주문, 핰, 늦, 시간, 시키, 렄, 국물, 걸쭉하,...","[('저번', 0.6136), ('주문', 0.5581), ('국물', 0.5323..."
37136,맛있어요 또시킬게요,고객님 안녕하요 짬뽕관입니다! 소중한 리뷰 남겨주셔 서 정말 달감사합니다! 만족스러...,5,"[맛있, 시키]","[('시키', 0.9276), ('맛있', 0.5392)]"
37137,정말 맛있게 잘먹었어요!자주 시켜 먹을께요!,고객님 안녕하세요 짬뽕관입니다~ 맛있게 드셨다는 말씀에 힘이 나는거 같습니다 항상 ...,5,"[맛있, 먹, 시키, 먹]","[('시키', 0.6619), ('맛있', 0.4214)]"
37138,와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...,고객님 안녕하세요 짬뽕관입니다! 맛있게 드셔주시고 정성스러운 사진 리뷰까지 정말 감...,5,"[주변, 맛있, 먹, 하, 시키, 육수, 시원, 칼칼하, 조개, 맛, 나, 좋, 덮...","[('칼칼하', 0.5926), ('들어오', 0.4927), ('신선', 0.48..."


In [ ]:
# train.drop(columns = 'keybert_reviews',axis=1)

In [ ]:
import re

total_keybert = []

for i in train['keybert_reviews']:
    korean_words = re.findall(r'[가-힣]+', i)  # ' ' 사이의 한글 단어 추출
    total_keybert.extend(korean_words)


In [ ]:
len(total_keybert) #총 추출 키워드 101698

101698

In [ ]:
total_keybert[:10]

['묵사발', '시원', '맛있', '생각', '기름', '부분', '청국장', '푸짐', '냄새', '맛있']

- Keybert 추출 결과의 Counter 탑 200(or 100) 중 라벨 키워드 각각 확인

In [ ]:
from collections import Counter

word_counts = Counter(total_keybert)

sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

for word, count in sorted_word_counts[:200]:
    print(f"{word}: {count}")

맛있: 21875
시키: 4211
배달: 4101
주문: 2641
감사: 2154
빠르: 2126
다음: 1392
맛나: 1242
서비스: 1136
최고: 947
리뷰: 912
고기: 713
오늘: 701
처음: 681
깔끔: 657
만족: 601
사진: 583
푸짐: 545
피자: 521
번창: 501
음식: 466
생각: 458
요청: 445
국물: 435
그렇: 434
메뉴: 431
추천: 426
오랜만: 416
괜찮: 412
사장: 405
추가: 378
김치: 354
소스: 336
떡볶이: 330
포장: 328
초밥: 325
친절: 322
가성비: 319
시간: 318
배부르: 308
가격: 305
반찬: 298
이벤트: 298
치킨: 294
푸짐하: 289
생각나: 285
치즈: 285
드리: 277
마라탕: 267
맛집: 254
커피: 242
돈까스: 232
아이: 226
매콤하: 225
적당: 220
토핑: 219
족발: 218
정도: 213
사항: 213
이번: 212
느낌: 209
냉면: 207
좋아하: 206
배송: 204
보쌈: 203
곱창: 200
구성: 196
느끼: 188
따뜻: 178
신선: 176
파스타: 174
모르: 171
양도: 169
땡기: 166
단골: 163
계란: 159
김밥: 157
보내: 150
주먹밥: 150
매장: 150
냄새: 149
넉넉: 148
비빔밥: 148
스파게티: 144
새우: 140
탕수육: 140
들어가: 137
미치: 136
이용: 133
다양: 130
다르: 129
샐러드: 127
짬뽕: 125
튀김: 123
양념: 122
가게: 121
닭발: 121
세트: 120
재료: 120
꿔바로우: 120
연어: 120
입맛: 119
덮밥: 118
전화: 110
정성: 110
기분: 109
만들: 107
매콤: 106
혼자: 106
덕분: 106
김치찌개: 104
신선하: 104
배고프: 103
두툼: 103
도착: 103
식사: 103
선택: 102
남기: 102
이상: 101
건강: 100
부르: 

In [ ]:
total_keybert = list(set(total_keybert))
len(total_keybert) #5837개

5837

In [ ]:
#200위 내 키워드들 중에 맛, 양, 가격, 서비스, 배달 관련 키워드 추출
taste_keyword = ['맛있','맛나','맛집','매콤하','느끼','신선','냄새','입맛','신선하','건강','매콤','싱싱하','잡내','매운맛','달달하']
quantity_keyword = ['양','푸짐','푸짐하','양도','넉넉','다양','든든','두툼','부족','배부르','든든하'] #'양'추가함
price_keyword = ['가성비','가격','비싸','저렴하']
service_keyword = ['서비스','깔끔','이벤트','친절']
delivery_keyword = ['배달','빠르','배송','도착']


# #100위 내 키워드들 중 맛, 양, 가격, 서비스, 배달 관련 키워드 추출
# taste_keyword = ['맛있','맛나','맛집','매콤하','신선하','냄새']
# quantity_keyword = ['양','푸짐','푸짐하','넉넉','다양','두툼','배부르'] #'양'추가함
# price_keyword = ['가성비','가격']
# service_keyword = ['서비스','깔끔','이벤트','친절']
# delivery_keyword = ['배달','빠르','배송']

In [ ]:
train['category'] = None

In [ ]:
def get_category(words):
    found_categories = set()  # 중복을 제외한 카테고리를 저장할 집합

    for kiwi in words:
        if kiwi in taste_keyword:
            found_categories.add('맛')
        elif kiwi in quantity_keyword:
            found_categories.add('양')
        elif kiwi in price_keyword:
            found_categories.add('가격')
        elif kiwi in service_keyword:
            found_categories.add('서비스')
        elif kiwi in delivery_keyword:
            found_categories.add('배달')

    # 집합을 리스트로 변환
    unique_categories = list(found_categories)

    return unique_categories


In [ ]:

# 'get_category' 함수를 'apply' 함수와 람다 함수를 사용하여 'category' 열에 적용
train['category'] = train['kiwi_reviews'].apply(lambda words: get_category(words))


In [ ]:
train

,review,reply,rating,kiwi_reviews,keybert_reviews,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,"[맛있, 먹, 묵사발, 시원, 맛있]","[('묵사발', 0.8007), ('시원', 0.6677), ('맛있', 0.4334)]",[맛]
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,"[기름, 있, 부분, 생각, 많, 참, 하]","[('생각', 0.7255), ('기름', 0.5717), ('부분', 0.5354)]",[]
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,"[양, 푸짐, 맛있, 청국장, 냄새, 나, 콩, 들, 좋]","[('청국장', 0.6928), ('푸짐', 0.3954), ('냄새', 0.315...","[양, 맛]"
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5,[],[],[]
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5,"[냉면, 시키, 미니, 냉면, 합치, 양, 적, 보이, 김치, 정식, 양, 적, 보...","[('정식', 0.4585), ('시키', 0.3429), ('합치', 0.323)...",[양]
...,...,...,...,...,...,...
37135,맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...,고객님 안녕하세요 짬뽕관입니다~ 소중한 리뷰 너무 감사합니다! 재주문해 주셨는데 국...,5,"[맛있, 저번, 주문, 맛있, 주문, 핰, 늦, 시간, 시키, 렄, 국물, 걸쭉하,...","[('저번', 0.6136), ('주문', 0.5581), ('국물', 0.5323...",[맛]
37136,맛있어요 또시킬게요,고객님 안녕하요 짬뽕관입니다! 소중한 리뷰 남겨주셔 서 정말 달감사합니다! 만족스러...,5,"[맛있, 시키]","[('시키', 0.9276), ('맛있', 0.5392)]",[맛]
37137,정말 맛있게 잘먹었어요!자주 시켜 먹을께요!,고객님 안녕하세요 짬뽕관입니다~ 맛있게 드셨다는 말씀에 힘이 나는거 같습니다 항상 ...,5,"[맛있, 먹, 시키, 먹]","[('시키', 0.6619), ('맛있', 0.4214)]",[맛]
37138,와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...,고객님 안녕하세요 짬뽕관입니다! 맛있게 드셔주시고 정성스러운 사진 리뷰까지 정말 감...,5,"[주변, 맛있, 먹, 하, 시키, 육수, 시원, 칼칼하, 조개, 맛, 나, 좋, 덮...","[('칼칼하', 0.5926), ('들어오', 0.4927), ('신선', 0.48...","[양, 서비스, 맛]"


- 카테고리 분류된 데이터들만 추림 (약 30000개)

In [ ]:
train_ft = train[train['category'].apply(lambda x: x != '[]' and len(x) > 0)]
train_ft.shape
train_ft

,review,reply,rating,kiwi_reviews,keybert_reviews,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,"[맛있, 먹, 묵사발, 시원, 맛있]","[('묵사발', 0.8007), ('시원', 0.6677), ('맛있', 0.4334)]",[맛]
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,"[양, 푸짐, 맛있, 청국장, 냄새, 나, 콩, 들, 좋]","[('청국장', 0.6928), ('푸짐', 0.3954), ('냄새', 0.315...","[양, 맛]"
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5,"[냉면, 시키, 미니, 냉면, 합치, 양, 적, 보이, 김치, 정식, 양, 적, 보...","[('정식', 0.4585), ('시키', 0.3429), ('합치', 0.323)...",[양]
5,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께 해주셔서 영광입니다. 다음에도 고객님의 즐거운 순...,5,"[맛나, 묵]","[('맛나', 0.5091)]",[맛]
6,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,"[맛, 양, 만족, 배달, 보내, 만족]","[('만족', 0.861), ('보내', 0.6242), ('배달', 0.5752)]","[배달, 양]"
...,...,...,...,...,...,...
37135,맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...,고객님 안녕하세요 짬뽕관입니다~ 소중한 리뷰 너무 감사합니다! 재주문해 주셨는데 국...,5,"[맛있, 저번, 주문, 맛있, 주문, 핰, 늦, 시간, 시키, 렄, 국물, 걸쭉하,...","[('저번', 0.6136), ('주문', 0.5581), ('국물', 0.5323...",[맛]
37136,맛있어요 또시킬게요,고객님 안녕하요 짬뽕관입니다! 소중한 리뷰 남겨주셔 서 정말 달감사합니다! 만족스러...,5,"[맛있, 시키]","[('시키', 0.9276), ('맛있', 0.5392)]",[맛]
37137,정말 맛있게 잘먹었어요!자주 시켜 먹을께요!,고객님 안녕하세요 짬뽕관입니다~ 맛있게 드셨다는 말씀에 힘이 나는거 같습니다 항상 ...,5,"[맛있, 먹, 시키, 먹]","[('시키', 0.6619), ('맛있', 0.4214)]",[맛]
37138,와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...,고객님 안녕하세요 짬뽕관입니다! 맛있게 드셔주시고 정성스러운 사진 리뷰까지 정말 감...,5,"[주변, 맛있, 먹, 하, 시키, 육수, 시원, 칼칼하, 조개, 맛, 나, 좋, 덮...","[('칼칼하', 0.5926), ('들어오', 0.4927), ('신선', 0.48...","[양, 서비스, 맛]"


- TF-IDF 추출

In [ ]:
# train['review'].tolist()

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# # 텍스트 데이터
# corpus = train['review'].tolist()

# # TF-IDF 벡터화 객체 생성
# tfidf_vectorizer = TfidfVectorizer()

# # TF-IDF 벡터 생성
# tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# # TF-IDF 행렬을 배열로 변환
# tfidf_array = tfidf_matrix.toarray()

# # 단어 목록 가져오기
# words = tfidf_vectorizer.get_feature_names_out()

# # TF-IDF 결과 출력
# for i, doc in enumerate(corpus):
#     print(f"Document {i + 1}:")
#     for j, word in enumerate(words):
#         print(f"{word}: {tfidf_array[i][j]:.2f}")
#     print()


In [ ]:
# tfidf_df = pd.DataFrame(data=tfidf_array, columns=words)
# # Add the 'tfidf' column to the 'train' DataFrame
# train['tfidf'] = tfidf_df.values.tolist()

In [ ]:
train

,review,reply,rating,kiwi_reviews,keybert_reviews,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,"[맛있, 먹, 묵사발, 시원, 맛있]","[('묵사발', 0.8007), ('시원', 0.6677), ('맛있', 0.4334)]",[맛]
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,"[기름, 있, 부분, 생각, 많, 참, 하]","[('생각', 0.7255), ('기름', 0.5717), ('부분', 0.5354)]",[]
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,"[양, 푸짐, 맛있, 청국장, 냄새, 나, 콩, 들, 좋]","[('청국장', 0.6928), ('푸짐', 0.3954), ('냄새', 0.315...","[양, 맛]"
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5,[],[],[]
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5,"[냉면, 시키, 미니, 냉면, 합치, 양, 적, 보이, 김치, 정식, 양, 적, 보...","[('정식', 0.4585), ('시키', 0.3429), ('합치', 0.323)...",[양]
...,...,...,...,...,...,...
37135,맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...,고객님 안녕하세요 짬뽕관입니다~ 소중한 리뷰 너무 감사합니다! 재주문해 주셨는데 국...,5,"[맛있, 저번, 주문, 맛있, 주문, 핰, 늦, 시간, 시키, 렄, 국물, 걸쭉하,...","[('저번', 0.6136), ('주문', 0.5581), ('국물', 0.5323...",[맛]
37136,맛있어요 또시킬게요,고객님 안녕하요 짬뽕관입니다! 소중한 리뷰 남겨주셔 서 정말 달감사합니다! 만족스러...,5,"[맛있, 시키]","[('시키', 0.9276), ('맛있', 0.5392)]",[맛]
37137,정말 맛있게 잘먹었어요!자주 시켜 먹을께요!,고객님 안녕하세요 짬뽕관입니다~ 맛있게 드셨다는 말씀에 힘이 나는거 같습니다 항상 ...,5,"[맛있, 먹, 시키, 먹]","[('시키', 0.6619), ('맛있', 0.4214)]",[맛]
37138,와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...,고객님 안녕하세요 짬뽕관입니다! 맛있게 드셔주시고 정성스러운 사진 리뷰까지 정말 감...,5,"[주변, 맛있, 먹, 하, 시키, 육수, 시원, 칼칼하, 조개, 맛, 나, 좋, 덮...","[('칼칼하', 0.5926), ('들어오', 0.4927), ('신선', 0.48...","[양, 서비스, 맛]"


- 네이버 라벨 활용 추출

In [ ]:
# taste_keywords = ['맛','반찬','사이드','안주','건강', '향신료', '재료', '잡내', '질']
# quantity_keywords = ['양','알차']
# price_keywords = ['가격', '가성비', '비싼', '가치']
# service_keywords = ['친절','포장','빨리']

In [ ]:
# train['naver_category'] = None

In [ ]:
# for i, words in tqdm(enumerate(train['review'])):
#     found_categories = []  # 중복을 제외한 카테고리를 저장할 집합

#     for review in words:
#         if kiwi in taste_keyword:
#             found_categories.add('맛')
#         elif kiwi in quantity_keyword:
#             found_categories.add('양')
#         elif kiwi in price_keyword:
#             found_categories.add('가격')
#         elif kiwi in service_keyword:
#             found_categories.add('서비스')
#         elif kiwi in delivery_keyword:
#             found_categories.add('배달')

#     # if not found_categories:
#     #     found_categories.add('')  # Default to '서비스' if no category is found

#     # 집합을 리스트로 변환
#     # unique_categories = list(found_categories)
#     train.loc[i, 'naver_category'] = ', '.join(found_categories)  # 콤마로 구분하여 문자열로 변환하여 할당


## 키워드 라벨화

In [ ]:
train_ft[['review','category']]


,review,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,[맛]
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,"[양, 맛]"
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,[양]
5,맛나게 잘 묵었습니다,[맛]
6,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"[배달, 양]"
...,...,...
37135,맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...,[맛]
37136,맛있어요 또시킬게요,[맛]
37137,정말 맛있게 잘먹었어요!자주 시켜 먹을께요!,[맛]
37138,와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...,"[양, 서비스, 맛]"


## 모델 학습

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.preprocessing import LabelBinarizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier


In [ ]:
# train_ft['category'] = train_ft['category'].str.split(',')
# train_ft['category']

In [ ]:

# MultiLabelBinarizer 원핫 인코딩
mlb = MultiLabelBinarizer()
encoded_labels = mlb.fit_transform(train_ft['category'])
label_classes = mlb.classes_

In [ ]:
encoded_labels.shape

(29863, 5)

In [ ]:
encoded_labels  #train의 카테고리 부분을 모두 멀티라벨 태스크로 바꾼 것


array([[0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 1, 0, 0, 0],
       [0, 1, 0, 1, 1],
       [0, 0, 1, 0, 0]])

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100)
multi_output_classifier = MultiOutputClassifier(rf_classifier)

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier


from sklearn.neural_network import MLPClassifier

from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.pipeline import Pipeline

# 데이터 불러오기
# data = pd.read_csv('reviews.csv')  # 리뷰 데이터 파일을 불러옵니다.

# 데이터 전처리
# 리뷰 텍스트를 TF-IDF 벡터로 변환합니다.
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(train_ft['review'])

# 다중 라벨을 바이너리로 변환합니다.
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(train_ft['category'])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 다중분류 모델 정의
models = {
    "Logistic Regression": LogisticRegression(),
    "Linear Support Vector Machine": LinearSVC(),
    "Linear SVM": SVC(kernel="linear"),
    "Decision Trees": DecisionTreeClassifier(),
    "Random Forests": RandomForestClassifier(),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier(),
    "Multilayer Perceptron, MLP": MLPClassifier(),

    "Ensemble Models": VotingClassifier(estimators=[
        ('rf', RandomForestClassifier()),
        ('gb', GradientBoostingClassifier()),
        ('xgb', XGBClassifier())
    ], voting='soft'),
    "K-Nearest Neighbors, KNN": KNeighborsClassifier()
}



In [ ]:
for model_name, model in tqdm(models.items()):
    # 모델 학습
    classifier = OneVsRestClassifier(model)
    classifier.fit(X_train, y_train)

    # 모델 평가
    y_pred = classifier.predict(X_test)

    # F1 스코어와 정확도 계산
    f1_micro = f1_score(y_test, y_pred, average='micro')
    accuracy = accuracy_score(y_test, y_pred)

    # 결과 출력
    print(f"Model: {model_name}")
    print(f"F1 스코어 (Micro): {f1_micro:.2f}")
    print(f"정확도: {accuracy:.2f}")

    # 새로운 리뷰에 라벨 예측
    new_reviews = ["맛이 정말 좋아요!", "양이 부족해요.", "가격이 너무 비싸요.",'사장님이 친절해요','배달이 느려요']
    new_reviews_vectorized = tfidf_vectorizer.transform(new_reviews)
    predicted_labels = classifier.predict(new_reviews_vectorized)
    predicted_categories = mlb.inverse_transform(predicted_labels)

    for review, categories in zip(new_reviews, predicted_categories):
        print(f"리뷰: {review}")
        print(f"예측 라벨: {categories}")

    print("=" * 50)

# 18000개 데이터 25000개 데이터 -> score 비슷하나 예측라벨 퀄리티가 데이터 많을수록 더 좋음
# Model: Linear Support Vector Machine, RandomForest, LGBM 예측라벨 성능 괜찮음

#SVM 기준 : 14000 0.94/0.86  -> 20000 0.94/0.87 (0.01점 정도씩 상승) -> 30000 0.94 / 0.86


  0%|          | 0/10 [00:00<?, ?it/s]

Model: Logistic Regression
F1 스코어 (Micro): 0.92
정확도: 0.84
리뷰: 맛이 정말 좋아요!
예측 라벨: ('맛',)
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격', '맛')
리뷰: 사장님이 친절해요
예측 라벨: ('맛',)
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Linear Support Vector Machine
F1 스코어 (Micro): 0.94
정확도: 0.86
리뷰: 맛이 정말 좋아요!
예측 라벨: ()
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격',)
리뷰: 사장님이 친절해요
예측 라벨: ('맛', '서비스')
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Linear SVM
F1 스코어 (Micro): 0.94
정확도: 0.87
리뷰: 맛이 정말 좋아요!
예측 라벨: ()
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격',)
리뷰: 사장님이 친절해요
예측 라벨: ('맛',)
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Decision Trees
F1 스코어 (Micro): 0.92
정확도: 0.82
리뷰: 맛이 정말 좋아요!
예측 라벨: ('맛',)
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격',)
리뷰: 사장님이 친절해요
예측 라벨: ('맛',)
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Random Forests
F1 스코어 (Micro): 0.93
정확도: 0.86
리뷰: 맛이 정말 좋아요!
예측 라벨: ()
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격', '맛')
리뷰: 사장님이 친절해요
예측 라벨: ('맛',)
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: 

## MultiLabel Model (테스트)

In [ ]:
for model_name, model in tqdm(models.items()):

    classifier = OneVsRestClassifier(model)
    classifier.fit(X_train, y_train)

    # 모델 평가
    y_pred = classifier.predict(X_test)

    # F1 스코어와 정확도 계산
    f1_micro = f1_score(y_test, y_pred, average='micro')
    accuracy = accuracy_score(y_test, y_pred)

    # 결과 출력
    print(f"Model: {model_name}")
    print(f"F1 스코어 (Micro): {f1_micro:.2f}")
    print(f"정확도: {accuracy:.2f}")

    print('리뷰생성------')

    # 새로운 리뷰에 라벨 예측
    new_reviews = [
        '맛이 있어요',
        '최고에요',
        '사장님이 친절하시네요',
        '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
        '맛이 없어요',
        '이 머리카락 들어있는거 뭔가요?',
        '배달이 너무 불친절해요',
        '최악이에요',

        '이집 피자 최애 넘나 맛있어요',
        '치킨이 바삭바삭하니 맛있네요',
        '마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!',
        '간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^',
        '짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요',
        '쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요',
        '요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요',

        '닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍',
        '오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.',
        '크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!',
        '맛있게 먹었어요 4단계로 해도 맵지는 않아요',

        '배달 빠르게 와서 맛있게 먹었습니다!',
        '완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요',
        '맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지….',
        '……………..',
        '벌레 나왔어요.. 다신 안시켜먹을 것 같아요']

    new_reviews_vectorized = tfidf_vectorizer.transform(new_reviews)
    predicted_labels = classifier.predict(new_reviews_vectorized)
    predicted_categories = mlb.inverse_transform(predicted_labels)

    for review, categories in zip(new_reviews, predicted_categories):
        print(f"리뷰: {review}")
        print(f"예측 라벨: {categories}")

    print("=" * 70)


  0%|          | 0/10 [00:00<?, ?it/s]

Model: Logistic Regression
F1 스코어 (Micro): 0.92
정확도: 0.84
리뷰생성------
리뷰: 맛이 있어요
예측 라벨: ()
리뷰: 최고에요
예측 라벨: ()
리뷰: 사장님이 친절하시네요
예측 라벨: ('맛',)
리뷰: 맛은 있는데 배달이 식어서 왔어요ㅠㅠ
예측 라벨: ('배달',)
리뷰: 맛이 없어요
예측 라벨: ('맛',)
리뷰: 이 머리카락 들어있는거 뭔가요?
예측 라벨: ('맛',)
리뷰: 배달이 너무 불친절해요
예측 라벨: ('배달',)
리뷰: 최악이에요
예측 라벨: ('맛',)
리뷰: 이집 피자 최애 넘나 맛있어요
예측 라벨: ('맛',)
리뷰: 치킨이 바삭바삭하니 맛있네요
예측 라벨: ('맛',)
리뷰: 마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!
예측 라벨: ('맛', '양')
리뷰: 간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^
예측 라벨: ('맛', '양')
리뷰: 짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요
예측 라벨: ('배달',)
리뷰: 쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요
예측 라벨: ('맛',)
리뷰: 요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요
예측 라벨: ('맛',)
리뷰: 닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍
예측 라벨: ('맛',)
리뷰: 오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.
예측 라벨: ('맛', '양')
리뷰: 크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!
예측 라벨: ('맛',)
리뷰: 맛있게 먹었어요 4단계로 해도 맵지는 않아요
예측 라벨: ('맛',)
리뷰: 배달 빠르게 와서 맛있게 먹었습니다!
예측 라벨: ('맛', '배달')
리뷰: 완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요
예측 라벨: ('맛',)
리뷰: 맛은 정말 좋았지만, 편의점가

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model: Multilayer Perceptron, MLP
F1 스코어 (Micro): 0.92
정확도: 0.84
리뷰생성------
리뷰: 맛이 있어요
예측 라벨: ('서비스',)
리뷰: 최고에요
예측 라벨: ('양',)
리뷰: 사장님이 친절하시네요
예측 라벨: ('맛',)
리뷰: 맛은 있는데 배달이 식어서 왔어요ㅠㅠ
예측 라벨: ('배달',)
리뷰: 맛이 없어요
예측 라벨: ()
리뷰: 이 머리카락 들어있는거 뭔가요?
예측 라벨: ('맛',)
리뷰: 배달이 너무 불친절해요
예측 라벨: ('배달',)
리뷰: 최악이에요
예측 라벨: ('맛',)
리뷰: 이집 피자 최애 넘나 맛있어요
예측 라벨: ('맛',)
리뷰: 치킨이 바삭바삭하니 맛있네요
예측 라벨: ('맛',)
리뷰: 마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!
예측 라벨: ('맛', '양')
리뷰: 간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^
예측 라벨: ('맛', '양')
리뷰: 짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요
예측 라벨: ('배달',)
리뷰: 쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요
예측 라벨: ('맛',)
리뷰: 요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요
예측 라벨: ('맛',)
리뷰: 닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍
예측 라벨: ('맛',)
리뷰: 오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.
예측 라벨: ('맛', '양')
리뷰: 크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!
예측 라벨: ('맛',)
리뷰: 맛있게 먹었어요 4단계로 해도 맵지는 않아요
예측 라벨: ('가격', '맛')
리뷰: 배달 빠르게 와서 맛있게 먹었습니다!
예측 라벨: ('맛', '배달')
리뷰: 완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요
예측 라벨: ('맛',)
리뷰

# B. 멀티라벨 모델링

- faeture: review + reply + kiwi 함께 학습 :  점수는 상승했으나 예측 라벨 퀄리티는 낮아짐

In [ ]:
train_ft['len'] = train['review'].apply(len)

In [ ]:
train_ft

In [ ]:
train_ft.iloc[:,:4]

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier


from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.pipeline import Pipeline

# 데이터 불러오기
# data = pd.read_csv('reviews.csv')  # 리뷰 데이터 파일을 불러옵니다.

# 데이터 전처리
# 리뷰 텍스트를 TF-IDF 벡터로 변환합니다.
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# 텍스트 피처 추가
train_ft['combined_text'] = train_ft['review'] + ' '  + train_ft['reply'] + ' ' + train_ft['kiwi_reviews'].apply(lambda x: ' '.join(x))
# train_ft['combined_text'] = train_ft['review'] + ' ' + train_ft['kiwi_reviews'].apply(lambda x: ' '.join(x))
X = tfidf_vectorizer.fit_transform(train_ft['combined_text'])


# X_text = tfidf_vectorizer.fit_transform(train_ft['review'])


# 다중 라벨을 바이너리로 변환합니다.
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(train_ft['category'])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 다중분류 모델 정의
models = {
    "Logistic Regression": LogisticRegression(),
    "Linear Support Vector Machine": LinearSVC(),
    "Random Forests": RandomForestClassifier(),
    "LightGBM": LGBMClassifier(),
}



In [ ]:
for model_name, model in tqdm(models.items()):
    # 모델 학습
    classifier = OneVsRestClassifier(model)
    classifier.fit(X_train, y_train)

    # 모델 평가
    y_pred = classifier.predict(X_test)

    # F1 스코어와 정확도 계산
    f1_micro = f1_score(y_test, y_pred, average='micro')
    accuracy = accuracy_score(y_test, y_pred)

    # 결과 출력
    print(f"Model: {model_name}")
    print(f"F1 스코어 (Micro): {f1_micro:.2f}")
    print(f"정확도: {accuracy:.2f}")

    # 새로운 리뷰에 라벨 예측
    new_reviews = ["맛이 정말 좋아요!", "양이 부족해요.", "가격이 너무 비싸요.",'사장님이 친절해요','배달이 느려요']
    new_reviews_vectorized = tfidf_vectorizer.transform(new_reviews)
    predicted_labels = classifier.predict(new_reviews_vectorized)
    predicted_categories = mlb.inverse_transform(predicted_labels)

    for review, categories in zip(new_reviews, predicted_categories):
        print(f"리뷰: {review}")
        print(f"예측 라벨: {categories}")

    print("=" * 50)

# 18000개 데이터 25000개 데이터 -> score 비슷하나 예측라벨 퀄리티가 데이터 많을수록 더 좋음
# Model: Linear Support Vector Machine, RandomForest, LGBM 예측라벨 성능 괜찮음


## Feature Engineering

## 하이퍼파라미터 튜닝

# C. 멀티라벨 + 긍부정 분석

### 입력받으면, 문장 단위로 나누기 -> 문장 단위로 검사해서 카테고리에 넣기 -> 긍부정 확인 -> 각각의 카테고리가 몇 %의 긍부정을 가지고 있는지 확인하기

In [ ]:
review_list = [
        '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
        '맛은 있는데 배달이 너무 불친절해요',

        '최악이에요',
        '이집 피자 최애 넘나 맛있어요',
        '치킨이 바삭바삭하니 맛있네요',
        '마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!',
        '간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^',
        '짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요',
        '쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요',
        '요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요',

        '닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍',
        '오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.',
        '크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!',
        '맛있게 먹었어요 4단계로 해도 맵지는 않아요',

        '배달 빠르게 와서 맛있게 먹었습니다!',
        '완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요',
        '맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지….',
        '……………..',
        '벌레 나왔어요.. 다신 안시켜먹을 것 같아요'
]

In [ ]:
#긍부정 확인

        '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
        '맛은 있는데 배달이 너무 불친절해요',

        '최악이에요',
        '이집 피자 최애 넘나 맛있어요',
        '치킨이 바삭바삭하니 맛있네요',
        '마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!',
        '간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^',
        '짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요',
        '쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요',
        '요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요',

        '닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍',
        '오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.',
        '크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!',
        '맛있게 먹었어요 4단계로 해도 맵지는 않아요',

        '배달 빠르게 와서 맛있게 먹었습니다!',
        '완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요',
        '맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지….',
        '……………..',
        '벌레 나왔어요.. 다신 안시켜먹을 것 같아요'

# 자연어 분석의 4단계
1) 형태소 분석 (Morphological Analysis)
2) 구문 분석 (Syntax Analysis)
3) 의미 분석 (Semantic Analysis)
4) 화용 분석 (Pragmatic Analysis)

In [ ]:
import re

def split_reviews(review_list):
    split_reviews = []
    for review in review_list:
        # 정규 표현식을 사용하여 문장을 구문으로 분리
        sentences = re.split(r'[.!?]', review)
        # 공백 문자를 제거하고 빈 문자열을 제거
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        split_reviews.extend(sentences)
    return split_reviews

# 주어진 리뷰 리스트를 구문으로 분리
split_reviews = split_reviews(review_list)

# 분리된 리뷰 출력
for i, sentence in enumerate(split_reviews, start=1):
    print(f"{i}. {sentence}")

1. 맛은 있는데 배달이 식어서 왔어요ㅠㅠ
2. 맛은 있는데 배달이 너무 불친절해요
3. 최악이에요
4. 이집 피자 최애 넘나 맛있어요
5. 치킨이 바삭바삭하니 맛있네요
6. 마라샹궈 넘넘 맛있고 양도 푸짐해용
7. 감사합니다
8. 간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^
9. 짜장면 배달이 너무 느리게 와서 음식이 다 불었네요
10. 사장님 신경좀 써주세요
11. 쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요
12. 요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요
13. 닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍
14. 오늘은 비계가 많은거 같아요
15. 양이 상대적으로 적어서 아쉬웠어요
16. 크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요
17. 맛있게 먹었어요 4단계로 해도 맵지는 않아요
18. 배달 빠르게 와서 맛있게 먹었습니다
19. 완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면
20. 종종 시켜먹을게요
21. 맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요
22. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지…
23. ……………
24. 벌레 나왔어요
25. 다신 안시켜먹을 것 같아요


In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.8 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Kkma

# KoNLPy의 Kkma 분석기 인스턴스 생성
kkma = Kkma()

# 구문 분석할 한국어 텍스트
text = '맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지….'


# KoNLPy를 사용하여 문장 구문 분석 및 분리
sentences = kkma.sentences(text)

# 분리된 문장 출력
for i, sentence in enumerate(sentences, start=1):
    print(f"Sentence {i}: {sentence}")


Sentence 1: 맛은 정말 좋았지만, 편의점 가려 다 배달 도착한 걸 발견 못해 다면 다 상한 음식 먹을 뻔했네요.
Sentence 2: 노크도 없고 문자도 안주 시면 음식이 온지 어떻게 알으라는 건지….


In [ ]:
from konlpy.tag import Kkma

# KoNLPy의 Kkma 분석기 인스턴스 생성
kkma = Kkma()

# 구문 분석할 한국어 텍스트
text = review_list[8]

# KoNLPy를 사용하여 문장 구문 분석 및 분리
sentences = kkma.sentences(text)

# 분리된 문장 출력
for i, sentence in enumerate(sentences, start=1):
    print(f"Sentence {i}: {sentence}")


In [ ]:
import spacy

# spaCy 영어 모델 로드
nlp = spacy.load("en_core_web_sm")

# 구문 분석할 문장
sentence = review_list[8]

# spaCy를 사용한 구문 분석
doc = nlp(sentence)

# 의존 구문 분석 결과 출력
for token in doc:
    print(f"{token.text} --({token.dep_})--> {token.head.text}")


쌀국수 --(compound)--> 반품
먹는데 --(compound)--> 나왔네요
비닐 --(compound)--> 나왔네요
나왔네요 --(compound)--> 반품
반품 --(compound)--> 청결에
이런건 --(compound)--> 청결에
안하지만 --(compound)--> 청결에
청결에 --(nsubj)--> 신경
신경 --(ROOT)--> 신경
써주세요 --(acomp)--> 신경


In [ ]:
import nltk

nltk.download("punkt")  # 필요한 데이터 다운로드

from nltk import sent_tokenize

# 구문 분석할 텍스트
text = review_list[8]

# 문장 분리
sentences = sent_tokenize(text)

# 분리된 문장 출력
for i, sentence in enumerate(sentences, start=1):
    print(f"Sentence {i}: {sentence}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Sentence 1: 쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요


In [ ]:
def get_cat_posneg(review):
    # review를 구문/문장으로 분리한다


    # 구문/문장별 카테고리를 예측한다 (다중분류)


    # 각 구문/문장별 긍/부정을 예측한다


    #

In [ ]:
for review in review_list :
    if

In [ ]:
# @torch.no_grad()
# def chatbot(model1, model2, tokenizer, max_len, device):
#     model.eval()
#     while True:
#         text = input("user > ").strip()
#         if text == "quit":
#             break

#         # 긍/부정 분석
#         sentiment = analyze_reviews(text)
#         # print(sentiment)

#         # 키워드 분석
#         result = kw_model.extract_keywords(text,
#                               keyphrase_ngram_range=(1, 1),
#                               use_mmr=True,
#                               diversity=0.7,
#                               stop_words=None,
#                               top_n=3)
#         tmp = []
#         for i in result:
#             tmp.append(i[0])
#         keyword = listToString(tmp) # 리스트를 문자열로 변환

#         text = "" + keyword + ""
#         x = tokenizer.encode(text, return_tensors="pt").to(device)
#         q_len = len(text) + 1

#         # 긍/부정 다른 모델 적용
#         if sentiment == 1:
#             print("긍정")
#             result_ids = model1.generate(x,
#                                     max_length=max_len,
#                                     repetition_penalty=2.0,
#                                     use_cache=True,
#                                     do_sample=True,
#                                     temperature=0.8,
#                                     top_k=10,
#                                     )
#         else:
#             print("부정")
#             result_ids = model2.generate(x,
#                                     max_length=max_len,
#                                     repetition_penalty=2.0,
#                                     use_cache=True,
#                                     do_sample=True,
#                                     temperature=0.8,
#                                     top_k=10,
#                                     )

#         generated_text = tokenizer.decode(result_ids[0])

#         #맞춤법 검사
#         correct_spell(generated_text)

#         generated_text = generated_text[q_len:-4]

#         sentences = re.split(r'(?<=[.!?]) +', generated_text)
#         if len(sentences) > 3:
#             generated_text = ' '.join(sentences[:3])
#         else:
#             generated_text = ' '.join(sentences)

#         print("bot >", generated_text)
#         print("--------------------------------------------------------------------------------------------")



In [ ]:
train_ft[['review','category']]
